# Week 07 — Requests Session + Caching + Reproducibility

**Time budget:** ~2 hours  
**Goal:** Use sessions, headers, caching strategy, and reproducible runs; introduce pathlib and configs.

**Theme (PhD focus):** Human factors of privacy & security — scraping public pages (privacy policies, cookie notices, security help pages, standards/regulator guidance) and extracting *UX-relevant* signals.

---


## Responsible scraping note (important)
We will only scrape **public pages** and keep the volume small.
- Prefer a few pages, not thousands
- Respect robots.txt/Terms of Service when you scale later
- Avoid collecting personal data
- Add delays for politeness when doing multi-page work


## Setup
We’ll use `requests` + `BeautifulSoup`. Install if needed:

```bash
pip install requests beautifulsoup4 pandas matplotlib
```


In [ ]:
import re
import time
import json
from urllib.parse import urljoin, urlparse

import requests
from bs4 import BeautifulSoup

import pandas as pd
import matplotlib.pyplot as plt

## Reproducibility upgrades
Introduce:
- `requests.Session()` for consistent headers
- a tiny disk cache (save HTML) to avoid re-downloading
- run IDs / timestamps


### 🧠 Concept: The Session (`requests.Session`)

Imagine:
- `requests.get()` is like opening a **New Incognito Window** for every single click. No memory. No cookies.
- `session.get()` is like using your **Main Browser**. It remembers cookies, navigation history, and settings (headers).

Use `Session` for speed and consistency.

### 🧠 Concept: Caching (Time Travel)

Scraping is slow. The internet changes. Analysis is fast.
- **Problem**: If you re-run your code, you re-download everything. 100 pages = 2 minutes.
- **Solution**: Save the page to disk (`.html`).
- **Logic**: "If I have the file, read it. If I don't, download it."

This makes your research **Reproducible**. You can prove what the page looked like on Tuesday.

In [ ]:
from pathlib import Path
from datetime import datetime, timezone

CACHE_DIR = Path("cache_html")
CACHE_DIR.mkdir(exist_ok=True)

session = requests.Session()
session.headers.update({"User-Agent": "HF-PrivacyScraper/0.1"})

In [ ]:
def cache_key(url: str) -> str:
    safe = re.sub(r"[^a-zA-Z0-9]+", "_", url)[:120]
    return safe + ".html"

def fetch_with_cache(url: str, use_cache: bool = True) -> str:
    key = cache_key(url)
    path = CACHE_DIR / key
    if use_cache and path.exists():
        return path.read_text(encoding="utf-8", errors="ignore")
    r = session.get(url, timeout=20)
    r.raise_for_status()
    path.write_text(r.text, encoding="utf-8")
    return r.text

In [ ]:
url = "https://www.mozilla.org/en-US/privacy/"
html = fetch_with_cache(url, use_cache=True)
print("Cached bytes:", len(html))